In [13]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

from Dataset.economy import EconomyDataset
from Model.LSTM import LSTM

In [55]:
# 데이터 다운 & 전처리

df = pd.read_excel('../데이터/economy/economy_all.xlsx')
df = df['국고채금리'].values

In [56]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 2

train_dataset = EconomyDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = EconomyDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = EconomyDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

1
2


c:\Users\hkyoo\OneDrive\바탕 화면\SCI\코드\Dataset\economy.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  self.x = torch.FloatTensor(X)


In [57]:
for x,y in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  break

X 크기 : torch.Size([2, 5])
Y 크기 : torch.Size([2])


In [58]:
input_size = 5
hidden_size = 20
Model = LSTM(input_size, hidden_size)

In [59]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,input_size))
model(basic_data)


cpu is available
작동하는지 실험


tensor([-0.0892], grad_fn=<SelectBackward0>)

In [60]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [62]:
train_n = len(train_dataset)
val_n = len(val_dataset)

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        inputs = inputs.to(device)
        labels = labels.to(device)
        out = model(inputs)
        loss = criterion(out, labels)

        optimizer.zero_grad() #
        loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss.item() # 한 배치의 loss 더해주고,


    print(running_loss/train_n)

    with torch.no_grad():
              print("Calculating validation results...")
              model.eval()
              val_loss = 0
              for inputs, labels in val_loader:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outs = model(inputs)

                  loss= criterion(outs, labels)
                  val_loss += loss.item()

              print("val")
              print(
                  f"loss: {val_loss/val_n:4.8} || "
              )
    print('======================================================')


epoch : 0
train
0.6340218337959257
Calculating validation results...
val
loss: 3.2874574 || 
epoch : 1
train
0.11565458773864144
Calculating validation results...
val
loss: 1.7401589 || 
epoch : 2
train


c:\Users\hkyoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0.06687821526605242
Calculating validation results...
val
loss: 1.4350601 || 
epoch : 3
train
0.06432110759324759
Calculating validation results...
val
loss: 1.3383809 || 
epoch : 4
train
0.058081054731105565
Calculating validation results...
val
loss: 1.2470174 || 
